In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

In [3]:
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [26]:
stk_object.dt = 30
stk_object.Compute_AzEl()
AER_Data = stk_object.Get_Access_DP(stk_object.targets,
                                    stk_object.satellites,
                                    "AER Data/Default",
                                    ['Access Number','Time','Azimuth','Elevation'])
Cross_Track = stk_object.Get_Access_DP(stk_object.satellites,
                                       stk_object.targets,
                                       "Sat Angles Data",
                                       ["Cross Track"])

- Computing_AzEl [==========] 150/150 [100%] in 3.1s (48.82/s)                  ▃▁▃ 40/150 [27%] in 1s (~2s, 47.5/s) [==>       ] ▁▃▅ 46/150 [31%] in 1s (~2s, 48.2/s) [=========>] ▃▅▇ 142/150 [95%] in 3s (~0s, 48.7/s) 
- Computing_AER Data/Default [==========] 150/150 [100%] in 3.1s (48.49/s)      ===>      ] ▅▇▇ 50/150 [33%] in 1s (~2s, 44.3/s) 
- Computing_Sat Angles Data [==========] 150/150 [100%] in 1.3s (118.46/s)               ] ▃▅▇ 19/150 [13%] in 0s (~1s, 114.1/s) 


In [27]:
Planning_Data = {key:[] for key in AER_Data[0][0]}
Planning_Data['Satellite'] = []
Planning_Data['Target'] = []
Planning_Data['Cross Track'] = []

tar_bin_count = np.zeros([len(stk_object.targets),36,9])
for tar_num,tar in enumerate(stk_object.targets):
    data_holder = AER_Data[tar_num].copy()
    for sat_num,df in enumerate(data_holder):
        df['Elevation'] = np.abs(df['Elevation'])
        df['Target'] = tar_num*np.ones_like(df[list(df.keys())[0]]).astype(int)
        df['Satellite'] = sat_num*np.ones_like(df[list(df.keys())[0]]).astype(int)
        df['Cross Track'] = Cross_Track[sat_num][tar_num]['Cross Track']
        for key in Planning_Data:
            Planning_Data[key].extend(df[key])

bins = []
for idx in range(len(Planning_Data['Target'])):
    az = Planning_Data['Azimuth'][idx]
    el = Planning_Data['Elevation'][idx]
    bins.append(int(az//10*9+el//10))

Planning_Data["Bin Number"] = bins
Planning_Data = pd.DataFrame(Planning_Data).sort_values(by="Time")

In [28]:
hash_map = {}
for tar_num in range(len(stk_object.targets)):
    hash_map[tar_num] = {}
    tar_window = Planning_Data['Target'].values==tar_num
    for bin_num in range(324):
        bin_window = Planning_Data['Bin Number'].values==bin_num
        hash_map[tar_num][bin_num] = np.array([Planning_Data['Time'].values[bin_window&tar_window],
                                               Planning_Data['Satellite'].values[bin_window&tar_window].astype(int),
                                               Planning_Data['Cross Track'].values[bin_window&tar_window]]).T
percentages = [len(np.unique(Planning_Data[Planning_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
for tar_num,percentage in enumerate(percentages):
    print(f"Target_{tar_num} : {percentage:.2f}%")
print(f"Total : {sum(percentages)/len(percentages):.2f}%")

Target_0 : 64.51%
Target_1 : 73.77%
Target_2 : 61.73%
Target_3 : 70.06%
Target_4 : 68.21%
Target_5 : 66.98%
Target_6 : 57.72%
Target_7 : 83.64%
Target_8 : 65.74%
Target_9 : 64.20%
Target_10 : 62.35%
Target_11 : 63.58%
Target_12 : 62.96%
Target_13 : 70.99%
Target_14 : 67.90%
Total : 66.95%


In [236]:
def check_manueverability(time1,sat1,theta1,time2,sat2,theta2):
    dtheta = np.abs(theta2-theta1)
    dtime = time2-time1
    rate_cond = np.divide(dtheta,dtime,out=np.zeros_like(dtime),where=dtime!=0)<1
    sat_cond = sat1!=sat2
    return sat_cond|rate_cond

def get_earliest_available(chosen,data):
    available_indexes = np.ones_like(data[:,0])
    if len(data)>0:
        for tar_num in chosen.keys():
            for bin_num in chosen[tar_num].keys():
                feasibility = check_manueverability(chosen[tar_num][bin_num][0],chosen[tar_num][bin_num][1],chosen[tar_num][bin_num][2],data[:,0],data[:,1],data[:,2])
                available_indexes = available_indexes*feasibility
        available_indexes = list(available_indexes.astype(bool))
        if True in available_indexes:
            return data[available_indexes][0]
        else:
            return False
    else:
        return False

In [237]:
# Planning Data
chosen = {key:{} for key in range(len(stk_object.targets))}
bins = np.reshape([[[count,tar_num,bin_num] for bin_num,count in enumerate(tar_bin.ravel())] for tar_num,tar_bin in enumerate(stk_object.target_bins)],[len(stk_object.targets)*324,3]).astype(int)
for count,tar_num,bin_num in bins[bins[:,0].argsort()]:
    if count > 0:
        result = get_earliest_available(chosen,hash_map[tar_num][bin_num])
        if type(result)!=bool:
            chosen[tar_num][bin_num] = (result[0],result[1],result[2])

In [256]:
Times = []
Sats = []
Targets = []
Angles = []
for tar_num in chosen:
    for bin_num in chosen[tar_num]:
        Times.append(chosen[tar_num][bin_num][0])
        Sats.append(chosen[tar_num][bin_num][1])
        Angles.append(chosen[tar_num][bin_num][2])
        Targets.append(tar_num)

df = pd.DataFrame({"Time":Times,"Satellite":Sats,"Target":Targets,"Cross Track":Angles})
fig = px.scatter(df[df['Satellite'].values==0],x="Time",y="Target",hover_data="Cross Track")
fig.update_layout(title="Planned Data")
fig.update_xaxes(range = [650,2250])
fig.show()

In [257]:
fig = px.scatter(Planning_Data[Planning_Data['Satellite'].values==0],x="Time",y="Target",hover_data="Cross Track")
fig.update_layout(title="Original Data")
fig.update_xaxes(range = [650,2250])
fig.show()

In [244]:
percentages = [len(chosen[tar_num])/324*100 for tar_num in chosen]
times = [max([chosen[tar_num][bin_num][0] for bin_num in chosen[tar_num]]) for tar_num in chosen]
for tar_num in range(len(stk_object.targets)):
    print(f"Target_{tar_num} : {percentages[tar_num]:.2f}% in {times[tar_num]/3600:.2f} hours")
print(f"Average : {sum(percentages)/len(percentages):.2f}% in {sum(times)/len(times)/3600:.2f} hours")

Target_0 : 50.62% in 23.34 hours
Target_1 : 71.60% in 23.44 hours
Target_2 : 61.73% in 23.23 hours
Target_3 : 66.98% in 23.41 hours
Target_4 : 61.11% in 23.43 hours
Target_5 : 57.41% in 23.37 hours
Target_6 : 57.72% in 23.18 hours
Target_7 : 83.64% in 23.48 hours
Target_8 : 65.74% in 23.36 hours
Target_9 : 63.27% in 23.30 hours
Target_10 : 54.63% in 23.35 hours
Target_11 : 58.02% in 23.30 hours
Target_12 : 57.10% in 23.36 hours
Target_13 : 68.83% in 23.45 hours
Target_14 : 63.89% in 23.41 hours
Average : 62.82% in 23.36 hours
